In [2]:
import time

# Local time has date and time
t = time.localtime()
print('Local Time is ', t)

# Extract the time part
current_time = time.strftime("%S", t)
current_time = int(current_time)
current_time = current_time + 20
print('Current time is ', current_time, type(int(current_time)))

Local Time is  time.struct_time(tm_year=2024, tm_mon=3, tm_mday=1, tm_hour=20, tm_min=24, tm_sec=26, tm_wday=4, tm_yday=61, tm_isdst=0)
Current time is  46 <class 'int'>


In [48]:
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2
import pandas as pd
MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

def draw_landmarks_on_image(rgb_image, detection_result):
  hand_landmarks_list = detection_result.hand_landmarks
  handedness_list = detection_result.handedness
  annotated_image = np.copy(rgb_image)
  handedness_and_landmarks = {}
  handedness_and_landmarks[handedness_list[0][0].display_name] =  hand_landmarks_list[0]
  # Loop through the detected hands to visualize.
  for idx in range(len(hand_landmarks_list)):
    hand_landmarks = hand_landmarks_list[idx]
    handedness = handedness_list[idx]

    # Draw the hand landmarks.
    hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    hand_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      hand_landmarks_proto,
      solutions.hands.HAND_CONNECTIONS,
      solutions.drawing_styles.get_default_hand_landmarks_style(),
      solutions.drawing_styles.get_default_hand_connections_style())

    # Get the top left corner of the detected hand's bounding box.
    height, width, _ = annotated_image.shape
    x_coordinates = [landmark.x for landmark in hand_landmarks]
    y_coordinates = [landmark.y for landmark in hand_landmarks]
    text_x = int(min(x_coordinates) * width)
    text_y = int(min(y_coordinates) * height) - MARGIN

    # Draw handedness (left or right hand) on the image.
    cv2.putText(annotated_image, f"{handedness[0].category_name}",
                (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
                FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

  return annotated_image , handedness_and_landmarks

In [37]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2
import time
cap = cv2.VideoCapture(0)
base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options,
                                       num_hands=2)
detector = vision.HandLandmarker.create_from_options(options)
timeout = time.time() + 10
while cap.isOpened() and time.time()<=timeout:
    ret, frame = cap.read()

    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)

    results = detector.detect(mp_image)


    annotated_image , h_and_l = draw_landmarks_on_image(mp_image.numpy_view(), results)
    cv2.imshow('landmarks',cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
h_and_l

{'Right': [NormalizedLandmark(x=0.3361351490020752, y=0.7017923593521118, z=3.798945158450806e-07, visibility=0.0, presence=0.0),
  NormalizedLandmark(x=0.38743075728416443, y=0.6918421983718872, z=-0.037832099944353104, visibility=0.0, presence=0.0),
  NormalizedLandmark(x=0.4316449463367462, y=0.6621655225753784, z=-0.056464437395334244, visibility=0.0, presence=0.0),
  NormalizedLandmark(x=0.4693237245082855, y=0.6369134783744812, z=-0.07150352001190186, visibility=0.0, presence=0.0),
  NormalizedLandmark(x=0.5073952674865723, y=0.6307291388511658, z=-0.08638492971658707, visibility=0.0, presence=0.0),
  NormalizedLandmark(x=0.4290466010570526, y=0.5218981504440308, z=-0.038193631917238235, visibility=0.0, presence=0.0),
  NormalizedLandmark(x=0.4669448435306549, y=0.4439106285572052, z=-0.05750400945544243, visibility=0.0, presence=0.0),
  NormalizedLandmark(x=0.4890712797641754, y=0.40342235565185547, z=-0.07343629747629166, visibility=0.0, presence=0.0),
  NormalizedLandmark(x=0.

In [50]:
X = [ f'x_right_hand_{i}' for i in range(21)] + [f'x_left_hand_{i}' for i in range(21)]
Y = [f'y_right_hand_{i}' for i in range(21)] + [f'y_left_hand_{i}' for i in range(21)] 
Z = [f'z_right_hand_{i}' for i in range(21)] + [f'z_left_hand_{i}' for i in range(21)] 
FEATURE_COLUMNS = X + Y + Z

In [56]:
if 'Right' in h_and_l.keys():
    x_right_hand = [ coords.x for coords in  h_and_l['Right'] ] 
    y_right_hand = [ coords.y for coords in  h_and_l['Right'] ]
    z_right_hand = [ coords.z for coords in  h_and_l['Right'] ]
    x_left_hand = [None]*21
    y_left_hand = [None]*21
    z_left_hand = [None]*21
if 'left' in h_and_l.keys():
    x_left_hand = [ coords.x for coords in  h_and_l['Left'] ] 
    y_left_hand = [ coords.y for coords in  h_and_l['Left'] ]
    z_left_hand = [ coords.z for coords in  h_and_l['Left'] ]
    x_right_hand = [None]*21
    y_right_hand = [None]*21
    z_right_hand = [None]*21

x = x_right_hand + x_left_hand
y = y_right_hand + y_left_hand
z = z_right_hand + z_left_hand
hands = x + y + z
hands

[0.3361351490020752,
 0.38743075728416443,
 0.4316449463367462,
 0.4693237245082855,
 0.5073952674865723,
 0.4290466010570526,
 0.4669448435306549,
 0.4890712797641754,
 0.5093095302581787,
 0.39287832379341125,
 0.4154841899871826,
 0.42631369829177856,
 0.4355689287185669,
 0.357055127620697,
 0.3622390627861023,
 0.36635398864746094,
 0.3696967363357544,
 0.3229987323284149,
 0.3235151767730713,
 0.3233805298805237,
 0.3268600106239319,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 0.7017923593521118,
 0.6918421983718872,
 0.6621655225753784,
 0.6369134783744812,
 0.6307291388511658,
 0.5218981504440308,
 0.4439106285572052,
 0.40342235565185547,
 0.3639422059059143,
 0.49484825134277344,
 0.4025658369064331,
 0.34794604778289795,
 0.2995072603225708,
 0.4903540015220642,
 0.4057014584541321,
 0.35161635279655457,
 0.3065229058265686,
 0.5030913352966309,
 0.4326533079147339,
 0.389

In [55]:
hands_df = pd.DataFrame([hands] , columns=FEATURE_COLUMNS)
hands_df

,x_right_hand_0,x_right_hand_1,x_right_hand_2,x_right_hand_3,x_right_hand_4,x_right_hand_5,x_right_hand_6,x_right_hand_7,x_right_hand_8,x_right_hand_9,...,z_left_hand_11,z_left_hand_12,z_left_hand_13,z_left_hand_14,z_left_hand_15,z_left_hand_16,z_left_hand_17,z_left_hand_18,z_left_hand_19,z_left_hand_20
0,0.336135,0.387431,0.431645,0.469324,0.507395,0.429047,0.466945,0.489071,0.50931,0.392878,...,None,None,None,None,None,None,None,None,None,None
